## 5.10 Homework

In this homework, we'll use the churn prediction model trained on a smaller set of features.

## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [1]:
!pip install pipenv

  Using cached pipenv-2021.5.29-py2.py3-none-any.whl (3.9 MB)
  Using cached virtualenv_clone-0.5.7-py3-none-any.whl (6.6 kB)


In [4]:
!pipenv --version

pipenv, version 2021.5.29


## Question 2

* Use Pipenv to install Scikit-Learn version 1.0
* What's the first hash for scikit-learn you get in Pipfile.lock? 

In [5]:
!pipenv install scikit-learn==1.0

Creating a Pipfile for this project...
Installing scikit-learn==1.0...
Adding scikit-learn to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock not found, creating...
Locking [dev-packages] dependencies...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (6c8284)!
Installing dependencies from Pipfile.lock (6c8284)...

To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


## Question 3

In [13]:
# pipenv shell
# pipenv run
%env PREFIX=https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework

!wget -O ../data/models/model1.bin $PREFIX/model1.bin
!wget -O ../data/models/dv.bin $PREFIX/dv.bin

env: PREFIX=https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework
--2021-10-10 16:32:35--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework/model1.bin
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[185.199.111.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 746 [application/octet-stream]
Guardando como: “../data/models/model1.bin”

../data/models/mode 100%[===================>]     746  --.-KB/s    en 0s      

2021-10-10 16:32:35 (34,5 MB/s) - “../data/models/model1.bin” guardado [746/746]

--2021-10-10 16:32:35--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework/dv.bin
Resolviendo raw.githubusercontent.com (raw.githubusercont

Let's use these models!

* Write a script for loading these models with pickle
* Score this customer:

```json
{"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}
```

What's the probability that this customer is churning? 

In [7]:
import pickle

In [12]:
with open('../data/models/dv.bin', 'rb') as file:
    dv = pickle.load(file)

/home/jloscalzo/.pyenv/versions/engagement_env/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [9]:
with open('../data/models/model1.bin', 'rb') as file:
    model = pickle.load(file)

/home/jloscalzo/.pyenv/versions/engagement_env/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [10]:
input_data = [{"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}]

In [11]:
model.predict_proba(dv.transform(input_data))

array([[0.88450419, 0.11549581]])

## Question 4

Now let's serve this model as a web service

* Install Flask and Gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this customer using `requests`:

```python
url = "YOUR_URL"
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
requests.post(url, json=customer).json()
```

What's the probability that this customer is churning?

In [17]:
# cd notebooks
# pipenv shell
# cd ..
# python scripts/homework5/serving.py
!pip install requests
import requests
import json

In [35]:
url = "http://localhost:1234/predict"
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}

requests.post(url, json=customer).json()

{'churn': True, 'churn_probability': 0.9988892771007961}

## Question 5

Now create your own Dockerfile based on the image I prepared.

It should start like that:

```docker
FROM agrigorev/zoomcamp-model:3.8.12-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with gunicorn 


When you build your image, what's the image id for `agrigorev/zoomcamp-model:3.8.12-slim`?

Look at the first step of your build log. It should look something like that:

```
$ docker some-command-for-building
Sending build context to Docker daemon  2.048kB
Step 1/N : FROM agrigorev/zoomcamp-model:3.8.12-slim
 ---> XXXXXXXXXXXX
Step 2/N : ....
```

You need this `XXXXXXXXXXXX`.

Alternatively, you can get this information when running `docker images` - it'll be in the "IMAGE ID" column.
Submitting DIGEST (long string starting with "sha256") is also fine.


# cd ../
# docker build --pull --rm -f "Dockerfile" -t mlzoomcamp2021:latest "."
# or
# docker build --pull --rm -f "Dockerfile" -t mlzoomcamp2021:latest "../"



## Question 6

Let's run your docker container!

After running it, score this customer:

```python
url = "YOUR_URL"
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}
requests.post(url, json=customer).json()
```

What's the probability that this customer is churning?

In [43]:
# docker run --rm -d  -p 1235:1235/tcp mlzoomcamp2021:latest
url = "http://localhost:1235/predict"
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}
requests.post(url, json=customer).json()

{'churn': False, 'churn_probability': 0.32940789808151005}